In [ ]:
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk

## In this i have done two sets mainly due to heavy training dataset.Due to huge training time i reduced the epoch to 1,so model didnt train well .As a result,the emotions are not cathing perfectly in the ouput.

##Second attempt done on selecting random samples from the dataset by balancing all the three sets.Eventhough this outperforms the previous,still have some issues on the output

Objective:
Develop machine learning models to classify emotions in text samples.
Dataset:
https://drive.google.com/file/d/1HWczIICsMpaL8EJyu48ZvRFcXx3_pcnb/view?usp=drive_link
Key components to be fulfilled :
1. Loading and Preprocessing (3 marks)
● Load the dataset and perform necessary preprocessing steps. This should include text
cleaning, tokenization, and removal of stopwords. Explain the preprocessing techniques
used and their impact on model performance.
2. Feature Extraction (2 marks):
● Implement feature extraction using CountVectorizer or TfidfVectorizer. Describe how the
chosen method transforms the text data into numerical features.
3. Model Development (2 marks):
● Train the following machine learning models
a)Naive Bayesb)Support Vector Machine

4. Model Comparison (2 marks)
● Evaluate the model using appropriate metrics (e.g., accuracy, F1-score). Provide a brief
explanation of the chosen model and its suitability for emotion classification.
5.Timely Submission (1 mark)
Submission Guidelines
● Provide your code in a Jupyter Notebook format and submit the GitHub link here.
● Ensure your explanations and answers are clear and concise.

In [ ]:
#Loading dataset
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1HWczIICsMpaL8EJyu48ZvRFcXx3_pcnb")
df.head()


,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear


Preprocessing

In [ ]:
df['Emotion'].value_counts()

,count
Emotion,
anger,4000000
joy,4000000
fear,3751969


In [ ]:
import re
def remove_tags(text):
    text = re.sub(r'<.*?>', '', text)           # remove HTML tags
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # remove URLs
    text = re.sub(r'[^\w\s]', ' ', text)        # remove punctuation
    text = text.lower()
    return text

df['Comment']=df['Comment'].apply(lambda cw : remove_tags(cw))

In [ ]:
#Removing stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['Comment'] = df['Comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Lemmatization (To perform this,we need wordnet corpus which usually finds the synonyms,antonyms etc)
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['Comment']=df['Comment'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# ----- Average Review Length (in words) -----
df['review_length'] = df['Comment'].apply(lambda x: len(str(x).split()))
average_length = df['review_length'].mean()
# ----- Emotion Percentage Distribution -----
emotion_percentages = df['Emotion'].value_counts(normalize=True) * 100

print("Average length of each review (in words): {:.2f}".format(average_length))
print("\nEmotion Distribution (%):")

for emotion, percentage in emotion_percentages.items():
    print(f"{emotion}: {percentage:.2f}%")


Average length of each review (in words): 9.38

Emotion Distribution (%):
anger: 33.69%
joy: 33.69%
fear: 32.63%


In [ ]:
label_encoder = LabelEncoder()
df['Emotion_label'] = label_encoder.fit_transform(df['Emotion'])
df=df.merge(df['Emotion_label'])
df.head()
df.count()

,0
Comment,11751969
Emotion,11751969
review_length,11751969
Emotion_label,11751969


In [ ]:
df['Emotion_label'].unique()
df['Emotion_label'].value_counts()

,count
Emotion_label,
0,4000000
2,4000000
1,3751969


train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['Comment'], df['Emotion_label'], test_size=0.2, random_state=42
)

In [ ]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 10 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [ ]:
from tensorflow import keras
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.build(input_shape=(None, max_length))
# model summary
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 10, 100)        │       300,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │         3,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 387,601 (1.48 MB)

 Trainable params: 387,601 (1.48 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
num_epochs = 1
history = model.fit(train_padded, y_train,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1)

264420/264420 ━━━━━━━━━━━━━━━━━━━━ 2151s 8ms/step - accuracy: 0.6212 - loss: -5205481.0000 - val_accuracy: 0.6371 - val_loss: -45275868.0000


In [ ]:
model_save=model.save("history.keras")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Predict probabilities on test data
prediction = model.predict(test_padded)

# Convert probabilities → class index
pred_classes = np.argmax(prediction, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test, pred_classes)
print("Accuracy on test set:", accuracy)

# Detailed classification report
print("\nClassification Report:\n")
print(classification_report(
    y_test,
    pred_classes,
    target_names=label_encoder.classes_
))

73450/73450 ━━━━━━━━━━━━━━━━━━━━ 200s 3ms/step
Accuracy on test set: 0.3396464592744876

Classification Report:



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       anger       0.34      1.00      0.51    798303
        fear       0.00      0.00      0.00    751151
         joy       0.00      0.00      0.00    800940

    accuracy                           0.34   2350394
   macro avg       0.11      0.33      0.17   2350394
weighted avg       0.12      0.34      0.17   2350394



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Sentence to predict
sentence = ["I am sitting alone scared"]

# Convert text → sequence
sequences = tokenizer.texts_to_sequences(sentence)

# Pad sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Predict probabilities
prediction = model.predict(padded)

# Get class index
pred_class = np.argmax(prediction, axis=1)

# Convert index → actual emotion label
pred_label = label_encoder.inverse_transform(pred_class)

print("Sentence:", sentence[0])
print("Predicted Emotion:", pred_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
Sentence: I am sitting alone scared
Predicted Emotion: anger


## Second Test

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1HWczIICsMpaL8EJyu48ZvRFcXx3_pcnb")

# due to large training dataset,for me its not working so reducing the training set
if len(df) > 100000:
    df = df.sample(n=100000, random_state=42).reset_index(drop=True)

# 2. Text Preprocessing
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'<.*?>', '', str(text))
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return " ".join(words)

df['Clean_Comment'] = df['Comment'].apply(clean_text)

# 3. Label Encoding
label_encoder = LabelEncoder()
df['Emotion_label'] = label_encoder.fit_transform(df['Emotion'])


# 4. Feature Extraction (Tokenization & Padding)
vocab_size = 5000
max_length = 50
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

X_train, X_test, y_train, y_test = train_test_split(
    df['Clean_Comment'], df['Emotion_label'], test_size=0.2, random_state=42
)

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# 5. Model Development
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 64, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dropout(0.2),
    # Use 3 units for Joy, Fear, Anger and 'softmax' for probability distribution
    keras.layers.Dense(3, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# 6. Training
num_epochs = 5
history = model.fit(
    train_padded, y_train,
    epochs=num_epochs,
    validation_split=0.1,
    batch_size=32
)

# 7. Evaluation
predictions = model.predict(test_padded)
pred_classes = np.argmax(predictions, axis=1)

print("\nAccuracy:", accuracy_score(y_test, pred_classes))
print("\nClassification Report:\n", classification_report(y_test, pred_classes, target_names=label_encoder.classes_))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.4015 - loss: 1.0661 - val_accuracy: 0.6042 - val_loss: 0.8347
Epoch 2/5
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8985 - loss: 0.3379 - val_accuracy: 0.9116 - val_loss: 0.2748
Epoch 3/5
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9761 - loss: 0.0803 - val_accuracy: 0.8989 - val_loss: 0.2952
Epoch 4/5
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9890 - loss: 0.0469 - val_accuracy: 0.9053 - val_loss: 0.2902
Epoch 5/5
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9962 - loss: 0.0253 - val_accuracy: 0.8968 - val_loss: 0.4508
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step

Accuracy: 0.8973063973063973

Classification Report:
               precision    recall  f1-score   support

       anger       0.92      0.84      0.88       392
        fear       0.88      0.94      0.91       416
         joy       0.90      0.91      0.90       380

    accuracy                           0.90      1188
   m

In [ ]:
def predict_emotion(text):
    # 1. Clean the input text
    cleaned_text = clean_text(text)
    # 2. Tokenize and pad (using the same tokenizer from training)
    sequence = tokenizer.texts_to_sequences([cleaned_text])
    padded = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    # 3. Predict
    prediction = model.predict(padded)
    # 4. Get the label with the highest probability
    class_index = np.argmax(prediction)
    emotion = label_encoder.inverse_transform([class_index])[0]
    # Optional: Print confidence scores
    print(f"\nText: {text}")
    print(f"Predicted Emotion: {emotion}")
    for i, prob in enumerate(prediction[0]):
        print(f"{label_encoder.classes_[i]}: {prob*100:.2f}%")

# Testing
predict_emotion("I am so happy that I passed the exam!")
predict_emotion("I heard a strange noise in the dark and got terrified.")
predict_emotion("It makes me so mad when people lie to me.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step

Text: I am so happy that I passed the exam!
Predicted Emotion: anger
anger: 90.08%
fear: 3.63%
joy: 6.29%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step

Text: I heard a strange noise in the dark and got terrified.
Predicted Emotion: fear
anger: 0.00%
fear: 100.00%
joy: 0.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step

Text: It makes me so mad when people lie to me.
Predicted Emotion: anger
anger: 99.30%
fear: 0.60%
joy: 0.10%
